In [ ]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.datasets import load_digits
from sklearn.datasets import load_boston

In [ ]:
def to_label(data, target, percentile):
    frac = percentile / 100.0
    part_val = data[target].quantile(frac)
    data[target] = [1 if d > part_val else 0 for d in data[target]]
    return data

In [99]:
#digits = load_digits()
boston = load_boston()
#data = pd.DataFrame(digits.data)
data = pd.DataFrame(boston.data, columns=boston.feature_names)
#label = 'class'
label = 'HomeVal50'
#data[label] = digits.target
data[label] = boston.target
to_label(data, label, 50)
data.head(5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,HomeVal50
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,1
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,1
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,1
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,1
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,1


In [105]:
def to_dict(df):
    '''
    Function to turn a dataframe into a dictionary
    '''
    # get the groups
    grouped = df.groupby(df.loc[:,label])
    classes = [k for k in grouped.groups.keys()]

    # for each class
    data_class = {}
    X = {}
    t = {}
    for k in classes:
        data_class[k] = grouped.get_group(k)
        t[k] = data_class[k][label]
        X[k] = data_class[k].drop(label,axis=1)
    return X, t, classes

In [106]:
X_dict, t, classes = to_dict(data)

### 2-class

In [ ]:
t = np.array(data[label])
X = data.drop(label, axis=1)

In [ ]:
N = X.shape[0]
D = X.shape[1]
print(N,D)

In [ ]:
# add a column of ones to X
ones = np.array([[1]*N]).T
phi = np.concatenate((ones,X), axis=1)
phi

In [ ]:
w = 0.001* np.random.randn(D+1)
a = np.dot(phi,w)

In [ ]:
a.shape

In [ ]:
def sigmoid(a):
    return 1/(1 + np.exp(-a))

In [ ]:
sigmoid(a)

In [ ]:
# Softmax function
def softmax(a):
    return np.exp(a) / float(sum(np.exp(a)))

In [ ]:
softmax(a)

### Multi-class

In [ ]:
w = {}
a = {}
for i in classes:
    w[i] =  0.001* np.random.randn(D+1)
    a[i] = np.dot(phi, w[i])

In [ ]:
def softmax(a_k):
    e_k = np.exp(a_k)
    e_total = np.sum([np.exp(a[i]) for i in classes])
    return e_k / e_total

In [ ]:
p = sigmoid(a)
grad = np.dot((p - t), phi)

In [ ]:
grad

In [ ]:
R = np.diag(p * (1 - p))

In [ ]:
z = np.dot(phi, w) - (np.dot((p-t), np.linalg.pinv(R)))

In [ ]:
np.linalg.inv(np.dot(np.dot(phi.T,R),phi)).dot(phi.T).dot(R).dot(z).shape

In [ ]:
z.shape

In [ ]:
np.zeros(D+1)

In [ ]:
np.dot(phi.T,phi).dot(w) - np.dot(phi.T,t)

In [ ]:
(p-t).dot(phi)

In [ ]:
t*np.log(2*p) + (1-t)*np.log(1-2*p)

In [ ]:
def safe_ln(p):
    for x in p:
        if x <= 0:
            return 0
        return np.log(x)

In [ ]:
-np.sum(t*safe_ln(p) + (1-t)*safe_ln(1-p))